In [1]:
from pathlib import Path
import fitz  # PyMuPDF
import os
from dotenv import load_dotenv

from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings


In [2]:
# ✅ Load environment variables
load_dotenv()
persist_dir = "../db"
pdf_dir = Path("../data/tutorials")  # Adjust path to your actual PDF folder

In [3]:
# ✅ Set up embedding model using HuggingFace
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/Volumes/My Volume/askpy-rag/ragenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# ✅ Extract text from a single PDF
def extract_text_from_pdf(pdf_path):
    print(f"📄 Reading: {pdf_path.name}")
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except Exception as e:
        print(f"❌ Failed to read {pdf_path.name}: {e}")
        return ""

In [5]:
from pathlib import Path

# ✅ Set path to folder containing PDFs
pdf_dir = Path("../data/tutorials")

# ✅ Load all .pdf files
pdf_files = list(pdf_dir.glob("*.pdf"))
if not pdf_files:
    raise FileNotFoundError("❌ No PDF files found in 'data/tutorials/'")

all_text = ""
for file in pdf_files:
    all_text += extract_text_from_pdf(file)

print("✅ All PDF content loaded.")


📄 Reading: Python Crash Course, 3rd Edition A Hands-On, Pr.pdf
📄 Reading: The Big Book Of Small Python Projects.pdf
📄 Reading: Python for data science 2022.pdf
📄 Reading: Automate_the_Boring_Stuff_with_Python,_2nd_Edition_@RedBlueHit.pdf
✅ All PDF content loaded.


In [6]:
# ✅ Chunk the text
splitter = CharacterTextSplitter(chunk_size=800, chunk_overlap=100)
chunks = splitter.split_text(all_text)
print(f"✅ Total chunks created: {len(chunks)}")

✅ Total chunks created: 1


In [ ]:
# ✅ Embed and store in Chroma
persist_dir = "../db"  # Go up one level from 'notebooks/' to the main folder
db = Chroma.from_texts(chunks, embedding, persist_directory=persist_dir)
print(f"✅ Vector database saved to '{persist_dir}'")

✅ Vector database saved to '../db'


/var/folders/cq/tvx_t3ws7xxcsv5lz1_j4j_40000gn/T/ipykernel_11110/1341724617.py:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()
